In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('default_of_credit_card_clients.xls')

In [3]:
import numpy as np

In [4]:
np.random.seed(seed=24)


In [5]:
random_integers = np.random.randint(low=1,high=5,size=100)


In [6]:
is_equal_to_3 = random_integers == 3


In [27]:
is_equal_to_3[:5]


array([ True, False, False, False, False])

In [28]:
sum(is_equal_to_3)


22

In [29]:
random_integers[is_equal_to_3]


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [30]:
id_counts = df['ID'].value_counts()
dupe_mask = id_counts == 2
dupe_mask[313:320]

3311e2e4-ee7f    False
2b6a5d78-2b8c    False
f75a8806-a2b5    False
8fe3647b-bb79    False
2b0b4e47-e32f    False
20823059-d5f3    False
c658ff8a-5878    False
Name: ID, dtype: bool

In [31]:
dupe_ids = id_counts.index[dupe_mask]
dupe_ids = list(dupe_ids)
len(dupe_ids)

313

In [32]:
df.loc[df['ID'].isin(dupe_ids[0:3]),:].head(10) 


,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
15687,183f18d5-0b44,10000,1,2,2,29,0,0,0,-1,...,6390,6682,7271,1000,0,6400,400,700,500,0
15787,183f18d5-0b44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26797,b46f955f-9e8a,280000,1,1,1,34,-1,-1,-1,-2,...,-21,15237,7345,76,0,0,15258,36,0,0
26897,b46f955f-9e8a,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28421,b557f3c2-ebec,130000,2,2,2,32,-2,-2,-2,-2,...,20739,7469,5963,14181,70505,20749,7469,5997,1978,0
28521,b557f3c2-ebec,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df_zero_mask = df == 0


In [34]:
feature_zero_mask = df_zero_mask.iloc[:,1:].all(axis=1)
sum(feature_zero_mask)


315

In [35]:
df_clean_1 = df.loc[~feature_zero_mask,:].copy()


In [38]:

df_clean_1['ID'].nunique()


29685

In [40]:
df_clean_1['PAY_1'].value_counts()

0                13087
-1                5047
1                 3261
Not available     3021
-2                2476
2                 2378
3                  292
4                   63
5                   23
8                   17
6                   11
7                    9
Name: PAY_1, dtype: int64

In [41]:
valid_pay_1_mask = df_clean_1['PAY_1'] != 'Not available'
valid_pay_1_mask[0:5]
sum(valid_pay_1_mask)

26664

In [43]:
df_clean_2 = df_clean_1.loc[valid_pay_1_mask,:].copy()
df_clean_2.shape

(26664, 25)

In [44]:
df_clean_2['PAY_1'] = df_clean_2['PAY_1'].astype('int64')
df_clean_2[['PAY_1', 'PAY_2']].info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 26664 entries, 0 to 29999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   PAY_1   26664 non-null  int64
 1   PAY_2   26664 non-null  int64
dtypes: int64(2)
memory usage: 624.9 KB


In [45]:
df_clean_2['EDUCATION'].value_counts()

2    12458
1     9412
3     4380
5      245
4      115
6       43
0       11
Name: EDUCATION, dtype: int64

In [46]:
df_clean_2['EDUCATION'].replace(to_replace=[0, 5, 6], value=4, inplace=True)
df_clean_2['EDUCATION'].value_counts()

2    12458
1     9412
3     4380
4      414
Name: EDUCATION, dtype: int64

In [47]:
df_clean_2['MARRIAGE'].replace(to_replace=0, value=3, inplace=True)
df_clean_2['MARRIAGE'].value_counts()

2    14158
1    12172
3      334
Name: MARRIAGE, dtype: int64

In [48]:
df_clean_2['EDUCATION_CAT'] = 'none'


In [49]:
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)


,EDUCATION,EDUCATION_CAT
0,2,none
1,2,none
2,2,none
3,2,none
4,2,none
5,1,none
6,1,none
7,2,none
8,3,none
9,3,none


In [50]:
cat_mapping = {
    1: "graduate school",
    2: "university",
    3: "high school",
    4: "others"
}

In [51]:
df_clean_2['EDUCATION_CAT'] = df_clean_2['EDUCATION'].map(cat_mapping)
df_clean_2[['EDUCATION', 'EDUCATION_CAT']].head(10)

,EDUCATION,EDUCATION_CAT
0,2,university
1,2,university
2,2,university
3,2,university
4,2,university
5,1,graduate school
6,1,graduate school
7,2,university
8,3,high school
9,3,high school


In [52]:
edu_ohe = pd.get_dummies(df_clean_2['EDUCATION_CAT'])
edu_ohe.head(10)
df_with_ohe = pd.concat([df_clean_2, edu_ohe], axis=1)
df_with_ohe[['EDUCATION_CAT', 'graduate school',
             'high school', 'university', 'others']].head(10)
df_with_ohe.to_csv('cleaned_data.csv', index=False)


In [53]:
edu_ohe.head(10)

,graduate school,high school,others,university
0,0,0,0,1
1,0,0,0,1
2,0,0,0,1
3,0,0,0,1
4,0,0,0,1
5,1,0,0,0
6,1,0,0,0
7,0,0,0,1
8,0,1,0,0
9,0,1,0,0
